In [21]:
%matplotlib inline
import csv
import pandas as pd
import scipy.sparse as sc
from helpers import *
from sgd import *
from als import *
from sklearn import decomposition
from scipy.optimize import minimize
from models import *
from plot import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
ratings = load_data('data_train.csv')

In [23]:
num_items_per_user = np.array((ratings != 0).sum(axis=0)).flatten()
num_users_per_item = np.array((ratings != 0).sum(axis=1).T).flatten()

In [24]:
_,train, test = split_data(ratings, num_items_per_user, num_users_per_item, 10, 0.1)

the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (9990, 999)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:1065253
Total number of nonzero elements in test data:111620


In [25]:
nnz_row, nnz_col = test.nonzero()
nnz_test = list(zip(nnz_row, nnz_col))

In [26]:
predict_item_test = np.array(baseline_item_mean(train,test))

test RMSE of the baseline using the item mean: [[ 1.09633198]].


In [27]:
predict_user_test = np.array(baseline_user_mean(train, test))

test RMSE of the baseline using the user mean: [[ 1.03317038]].


In [28]:
predict_global_test = baseline_global_mean(train, test)
predict_global_test

test RMSE of baseline using the global mean: [[ 1.12152228]].


3.8581107023810133

In [29]:
predict_SVD_test, rmse = svd_surprise(train, test, None, 10)

1.00165158065


In [30]:
predict_KNNitem_test, rmse = knn_surprise(train,test,None,False, 100)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
0.988904728304


In [31]:
predict_baseline_test = baseline_surprise(train,test,None)

Estimating biases using als...
1.00127314809


In [32]:
predict_slope_test = slope_surprise(train, test, None)

1.00185650691


In [33]:
%%time
predict_SGD_test, rmse = matrix_factorization_SGD(train, test, 0.025, 20, 0.1, 0.016)

RMSE on test data: 0.9837803281828068.
Wall time: 9min 36s


In [34]:
%%time

predict_ALS_test, rmse = ALS(train,test,3, 0.2, 0.9)


start the ALS algorithm...
RMSE on training set: 0.9683854696262746.
RMSE on training set: 0.9636641888328061.
RMSE on training set: 0.9622033086723247.
RMSE on training set: 0.9610708830609189.
RMSE on training set: 0.9600981287338146.
RMSE on training set: 0.9593451001369316.
RMSE on training set: 0.9588289719349584.
RMSE on training set: 0.9585034014048226.
RMSE on training set: 0.9583079650772224.
RMSE on training set: 0.958193546877028.
RMSE on training set: 0.9581268728266509.
test RMSE after running ALS: 0.9940086593042198.
Wall time: 4min 29s


In [35]:
all_predictions_test = []
all_predictions_test.append(predict_SGD_test)
all_predictions_test.append(predict_ALS_test)
all_predictions_test.append(predict_global_test)
all_predictions_test.append(predict_SVD_test)
all_predictions_test.append(predict_user_test)
all_predictions_test.append(predict_item_test.reshape(9990,1))
all_predictions_test.append(predict_KNNitem_test)
all_predictions_test.append(predict_baseline_test)
all_predictions_test.append(predict_slope_test)

In [36]:
def const(x):
    return x.sum() - 1
constraint = {'type':'eq', 'fun': const}

def sum_ratings(weights):
    mix_prediction = 0
    for i, pred in enumerate(all_predictions_test):
        mix_prediction += weights[i] * pred
    return compute_mix_error(test, nz=nnz_test, prediction=mix_prediction)


In [37]:
%%time
x0 = [1/len(all_predictions_test)] * len(all_predictions_test)
result = minimize(fun=sum_ratings, x0=x0)

Wall time: 8min 30s


In [38]:
result

      fun: 0.9783223030358202
 hess_inv: array([[  35.23846587,  -12.13447485,   -7.94560474,   -4.93093208,
           7.6837867 ,    7.3400074 ,  -10.11201621,   -1.59913887,
         -12.92650625],
       [ -12.13447485,   23.41640839,    1.87847135,   -5.5105406 ,
          -2.22198988,   -1.40539239,   -5.98282192,    5.17384502,
          -3.44702249],
       [  -7.94560474,    1.87847135,   87.70144834,    3.66840258,
         -88.9059799 ,  -84.97560251,    7.44913467,   82.9007929 ,
          -1.91244952],
       [  -4.93093208,   -5.5105406 ,    3.66840258,   27.97233023,
          -0.65865387,   -4.31658428,   -3.06356912,  -24.14321653,
          11.11479486],
       [   7.6837867 ,   -2.22198988,  -88.9059799 ,   -0.65865387,
         109.92433179,   88.83684505,   -5.75982423, -152.30516689,
          43.68141517],
       [   7.3400074 ,   -1.40539239,  -84.97560251,   -4.31658428,
          88.83684505,   87.83863784,   -7.29496764, -103.97172716,
          18.15659263],

In [39]:
%%time
predict_global = baseline_global_mean(ratings, None)
predict_user = np.array(baseline_user_mean(ratings, None))
predict_item = np.array(baseline_item_mean(ratings,None)).reshape(10000,1)
predict_SGD = matrix_factorization_SGD(ratings,None, 0.025, 20, 0.1, 0.016)
predict_ALS = ALS(ratings,None, 3, 0.2, 0.9)


Progression: 100.00%
start the ALS algorithm...
RMSE on training set: 0.9697751806835305.
RMSE on training set: 0.9652521967234385.
RMSE on training set: 0.9637298175089041.
RMSE on training set: 0.9625029911692241.
RMSE on training set: 0.96151340878739.
RMSE on training set: 0.9608201400665092.
RMSE on training set: 0.9603902940452437.
RMSE on training set: 0.9601438316572578.
RMSE on training set: 0.9600078549084466.
RMSE on training set: 0.9599332982742421.
Wall time: 15min 51s


In [40]:
to_predict = get_to_predict()
predict_SVD = svd_surprise(ratings, None, to_predict, 10 )

In [41]:
predict_KNNitem = knn_surprise(ratings, None, to_predict, False)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [42]:
predict_baseline = baseline_surprise(ratings, None, to_predict)

Estimating biases using als...


In [43]:
predict_slope = slope_surprise(ratings, None, to_predict)

In [44]:
all_predictions = []
all_predictions.append(predict_SGD)
all_predictions.append(predict_ALS)
all_predictions.append(predict_global)
all_predictions.append(predict_SVD)
all_predictions.append(predict_user)
all_predictions.append(predict_item.reshape(10000,1))
all_predictions.append(predict_KNNitem)
all_predictions.append(predict_baseline)
all_predictions.append(predict_slope)

In [45]:
final_predict = 0
for i, pred in enumerate(all_predictions):
    final_predict += result.x[i] * pred

In [46]:
final_predict[final_predict<1] = 1
final_predict[final_predict>5] = 5

In [47]:
create_csv_submissions(final_predict)